In [1]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.tail()

,Ticker
500,YUM
501,ZBH
502,ZBRA
503,ZION
504,ZTS


In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 90264964,
 'calculationPrice': 'close',
 'change': 6.58,
 'changePercent': 0.04244,
 'close': 168.88,
 'closeSource': 'lfaoifci',
 'closeTime': 1679434299760,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 171.07,
 'delayedPriceTime': 1720291292705,
 'extendedChange': -0.25,
 'extendedChangePercent': -0.00151,
 'extendedPrice': 168.78,
 'extendedPriceTime': 1658261946021,
 'high': 172.61,
 'highSource': 'duicl5dee pia1ryme  net',
 'highTime': 1725124873408,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 174.45,
 'iexCloseTime': 1670395386568,
 'iexLastUpdated': 1728632529584,
 'iexMarketPercent': 0.019380093980265606,
 'iexOpen': 166.74,
 'iexOpenTime': 1704772784428,
 'iexRealtimePrice': 170.85,
 'iexRealtimeSize': 6,
 'iexVolume': 2084221,
 'lastTradeTime': 1715333762634,
 'latestPrice': 170.87,
 'latestSource': 'Close',
 'latestTime': 'May 4, 2022',
 'latestUpdate': 1654121891706,
 'latestVolume': 10960202

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']
price, market_cap

(170.87, 2701818672141)

In [6]:
columns = ['Ticker', 'Stock Price', 'Market Cap.', 'Number of Shares to Buy']
final_df = pd.DataFrame(columns = columns)
final_df

,Ticker,Stock Price,Market Cap.,Number of Shares to Buy


In [7]:
final_df.append(
    pd.Series([symbol, price, market_cap, 'NaN' ], index=columns), ignore_index=True
)

/tmp/ipykernel_1203/3955257164.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df.append(


,Ticker,Stock Price,Market Cap.,Number of Shares to Buy
0,AAPL,170.87,2701818672141,NaN


In [9]:
# BATCH API CALL
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_df = pd.DataFrame(columns = columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
            pd.Series([symbol, data[symbol]['quote']['latestPrice'], data[symbol]['quote']['marketCap'], 'NaN'], index=columns), ignore_index=True
        )


/tmp/ipykernel_1203/2363738471.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/ipykernel_1203/2363738471.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/ipykernel_1203/2363738471.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/ipykernel_1203/2363738471.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/ipykernel_1203/2363738471.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.app

KeyError: 'DISCA'

In [10]:
final_df

,Ticker,Stock Price,Market Cap.,Number of Shares to Buy
0,A,126.6,38728544271,NaN
1,AAL,19.76,12763957039,NaN
2,AAP,212.75,13129712496,NaN
3,AAPL,169.24,2727841794624,NaN
4,ABBV,155.61,273976015858,NaN
...,...,...,...,...
130,DG,243.12,56141149542,NaN
131,DGX,143.55,16336906808,NaN
132,DHI,78.8,27607519492,NaN
133,DHR,266.29,189015895994,NaN


In [11]:
portfolio_size = input('Enter the value of your portfolio: ')
try:
    portfolio_size = float(portfolio_size)
except ValueError:
    print('Error: enter a int or float.')
print(portfolio_size)

10000000.0


In [12]:
position_size = portfolio_size/len(final_df.index)
for i in range(0, len(final_df.index)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i, 'Stock Price'])
final_df

,Ticker,Stock Price,Market Cap.,Number of Shares to Buy
0,A,126.6,38728544271,585
1,AAL,19.76,12763957039,3748
2,AAP,212.75,13129712496,348
3,AAPL,169.24,2727841794624,437
4,ABBV,155.61,273976015858,476
...,...,...,...,...
130,DG,243.12,56141149542,304
131,DGX,143.55,16336906808,516
132,DHI,78.8,27607519492,940
133,DHR,266.29,189015895994,278


In [13]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_df.to_excel(writer, 'Recommended Trades', index=False)

In [14]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1

    }
)
int_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,

    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1

    }
)

In [15]:
writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Cap.', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to buy', int_format)

writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
writer.save()